In [23]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [24]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [25]:
nodes = ['a','b','c','d','r']
edges_with_costs ={
    ('a','b'):3,('c','a'):1,('r','a'):9,('b','c'):8,
    ('b','r'):4,('d','b'):6,('r','c'):2,
    ('c','d'):7,('d','c'):5
}

In [26]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [27]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)

In [28]:
for v in nodes:
    if v != 'r':
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
        #prob +=pulp.lpSum(x[u,v] for u in nodes if (v,u) in edges_with_costs) == 1

Minimize_Cost:
MINIMIZE
3*x_a_b + 8*x_b_c + 4*x_b_r + 1*x_c_a + 7*x_c_d + 6*x_d_b + 5*x_d_c + 9*x_r_a + 2*x_r_c + 0
SUBJECT TO
_C1: x_c_a + x_r_a = 1

_C2: x_a_b + x_d_b = 1

_C3: x_b_c + x_d_c + x_r_c = 1

_C4: x_c_d = 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_b_r <= 1 Integer
0 <= x_c_a <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_d_b <= 1 Integer
0 <= x_d_c <= 1 Integer
0 <= x_r_a <= 1 Integer
0 <= x_r_c <= 1 Integer

In [30]:
prob += pulp.lpSum(x[u]['r'] for u in nodes if (u,'r') in edges_with_costs) == 0
prob += pulp.lpSum(x['r'][v] for v in nodes if ('r',v) in edges_with_costs)>= 1

Minimize_Cost:
MINIMIZE
3*x_a_b + 8*x_b_c + 4*x_b_r + 1*x_c_a + 7*x_c_d + 6*x_d_b + 5*x_d_c + 9*x_r_a + 2*x_r_c + 0
SUBJECT TO
_C1: x_c_a + x_r_a = 1

_C2: x_a_b + x_d_b = 1

_C3: x_b_c + x_d_c + x_r_c = 1

_C4: x_c_d = 1

_C5: x_b_r = 0

_C6: x_r_a + x_r_c >= 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_b_r <= 1 Integer
0 <= x_c_a <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_d_b <= 1 Integer
0 <= x_d_c <= 1 Integer
0 <= x_r_a <= 1 Integer
0 <= x_r_c <= 1 Integer

In [31]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if 'r' in s:
        subset.append(s)
subset

[('r',),
 ('a', 'r'),
 ('b', 'r'),
 ('c', 'r'),
 ('d', 'r'),
 ('a', 'b', 'r'),
 ('a', 'c', 'r'),
 ('a', 'd', 'r'),
 ('b', 'c', 'r'),
 ('b', 'd', 'r'),
 ('c', 'd', 'r'),
 ('a', 'b', 'c', 'r'),
 ('a', 'b', 'd', 'r'),
 ('a', 'c', 'd', 'r'),
 ('b', 'c', 'd', 'r')]

In [32]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1  


Minimize_Cost:
MINIMIZE
3*x_a_b + 8*x_b_c + 4*x_b_r + 1*x_c_a + 7*x_c_d + 6*x_d_b + 5*x_d_c + 9*x_r_a + 2*x_r_c + 0
SUBJECT TO
_C1: x_c_a + x_r_a = 1

_C2: x_a_b + x_d_b = 1

_C3: x_b_c + x_d_c + x_r_c = 1

_C4: x_c_d = 1

_C5: x_b_r = 0

_C6: x_r_a + x_r_c >= 1

_C7: x_r_a + x_r_c >= 1

_C8: x_a_b + x_r_c >= 1

_C9: x_b_c + x_r_a + x_r_c >= 1

_C10: x_c_a + x_c_d + x_r_a >= 1

_C11: x_d_b + x_d_c + x_r_a + x_r_c >= 1

_C12: x_b_c + x_r_c >= 1

_C13: x_a_b + x_c_d >= 1

_C14: x_a_b + x_d_b + x_d_c + x_r_c >= 1

_C15: x_c_a + x_c_d + x_r_a >= 1

_C16: x_b_c + x_d_c + x_r_a + x_r_c >= 1

_C17: x_c_a + x_d_b + x_r_a >= 1

_C18: x_c_d >= 1

_C19: x_b_c + x_d_c + x_r_c >= 1

_C20: x_a_b + x_d_b >= 1

_C21: x_c_a + x_r_a >= 1

VARIABLES
0 <= x_a_b <= 1 Integer
0 <= x_b_c <= 1 Integer
0 <= x_b_r <= 1 Integer
0 <= x_c_a <= 1 Integer
0 <= x_c_d <= 1 Integer
0 <= x_d_b <= 1 Integer
0 <= x_d_c <= 1 Integer
0 <= x_r_a <= 1 Integer
0 <= x_r_c <= 1 Integer

In [33]:
prob.solve()
print ("Status:", LpStatus[prob.status])

Status: Optimal


In [34]:
print ("Optimal Solution")
for i in nodes:
	for j in nodes:
		if x[i][j].value() == 1:
			print ("(%s,%s)"%(i,j),)

Optimal Solution
(a,b)
(c,a)
(c,d)
(r,c)
